<a href="https://colab.research.google.com/github/somkiatth/git/blob/master/kumllibAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 61.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f97ed6606db3f194f0a498e1ac3543758be86d5cff3fceac50bcf5ad147fae62
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
mldf=spark.read.csv("/content/census-income-all.csv", header=True, inferSchema=True)

In [ ]:
mldf.show(5)

+---+----------------+------+---------+-------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+
|age|       workclass|weight|education|edu num|    marital status|       occupation| relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|
+---+----------------+------+---------+-------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+
| 39|       State-gov| 77516|Bachelors|     13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States|   No|
| 50|Self-emp-not-inc| 83311|Bachelors|     13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States|   No|
| 38|         Private|215646|  HS-grad|      9|          Divorced|Handlers-cleaners|Not-in-family|White|  Male|           0|      

In [ ]:
cstrain,cstest = mldf.randomSplit([0.8,0.2])

In [ ]:
cstrain.count(),cstest.count()

(39096, 9746)

In [ ]:
cstrain.write.csv("/content/censustrain.csv")

In [ ]:
cstrainload=spark.read.load("/content/censustrain.csv",format='csv')

In [ ]:
cstrainload.count()

38986

In [ ]:
cstrain.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- edu num: integer (nullable = true)
 |-- marital status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- captial-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native country: string (nullable = true)
 |-- label: string (nullable = true)



In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
va=VectorAssembler(inputCols=['age','edu num','captial-gain','capital-loss','hours-per-week'],outputCol='features')

In [ ]:
vtraindf=va.transform(cstrain)

In [ ]:
vtraindf.show(5)

+---+---------+------+---------+-------+--------------+----------+------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+
|age|workclass|weight|education|edu num|marital status|occupation|relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|
+---+---------+------+---------+-------+--------------+----------+------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+
| 17|        ?| 27251|     11th|      7|       Widowed|         ?|   Own-child|White|  Male|           0|           0|            40| United-States|   No|[17.0,7.0,0.0,0.0...|
| 17|        ?| 34019|     10th|      6| Never-married|         ?|   Own-child|White|  Male|           0|           0|            20| United-States|   No|[17.0,6.0,0.0,0.0...|
| 17|        ?| 34088|     12th|      8| Never-married|         ?|   Own-child|White|Female|           0|           0|  

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
si=StringIndexer(inputCol='label',outputCol='label_idx',stringOrderType='frequencyDesc')

In [ ]:
labelindex = si.fit(vtraindf)

In [ ]:
vidxtraindf=labelindex.transform(vtraindf)

In [ ]:
vidxtraindf.show()

+---+---------+------+---------+-------+--------------+----------+--------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+
|age|workclass|weight|education|edu num|marital status|occupation|  relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|label_idx|
+---+---------+------+---------+-------+--------------+----------+--------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+
| 17|        ?| 27251|     11th|      7|       Widowed|         ?|     Own-child|White|  Male|           0|           0|            40| United-States|   No|[17.0,7.0,0.0,0.0...|      0.0|
| 17|        ?| 34019|     10th|      6| Never-married|         ?|     Own-child|White|  Male|           0|           0|            20| United-States|   No|[17.0,6.0,0.0,0.0...|      0.0|
| 17|        ?| 34088|     12th|      8| Never-married|     

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr=LogisticRegression(featuresCol='features',labelCol='label_idx',maxIter=1000, tol=.001,regParam=0)

In [ ]:
lrmodel = lr.fit(vidxtraindf)

In [ ]:
lrmodel.transform(vidxtraindf).show()

+---+---------+------+---------+-------+--------------+----------+--------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+--------------------+--------------------+----------+
|age|workclass|weight|education|edu num|marital status|occupation|  relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|label_idx|       rawPrediction|         probability|prediction|
+---+---------+------+---------+-------+--------------+----------+--------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+--------------------+--------------------+----------+
| 17|        ?| 27251|     11th|      7|       Widowed|         ?|     Own-child|White|  Male|           0|           0|            40| United-States|   No|[17.0,7.0,0.0,0.0...|      0.0|[3.70137397731913...|[0.97590530760920...|       0.0|
| 17|        ?| 34019|     10th|    

In [ ]:
lrmodel.coefficients

DenseVector([0.0427, 0.325, 0.0003, 0.0007, 0.0401])

In [ ]:
lrmodel.intercept

-8.306627004195706

In [ ]:
lrtrainresult=lrmodel.evaluate(vidxtraindf)

In [ ]:
lrtrainresult.accuracy

0.8143179602934387

In [ ]:
lrtrainresult.recallByLabel

[0.9481688810952991, 0.3889841405915131]

In [ ]:
lrtrainresult.labels

[0.0, 1.0]

In [ ]:
lrmodel.transform(vidxtraindf).sample(1/4000).show()

+---+----------------+------+------------+-------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+--------------------+--------------------+----------+
|age|       workclass|weight|   education|edu num|    marital status|       occupation| relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|label_idx|       rawPrediction|         probability|prediction|
+---+----------------+------+------------+-------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+--------------------+--------------------+----------+
| 17|         Private|154337|        11th|      7|     Never-married|    Other-service|    Own-child|White|Female|           0|           0|            13| United-States|   No|[17.0,7.0,0.0,0.0...|      0.0|[4.7849181

In [ ]:
vidxtestdf = labelindex.transform(va.transform(cstest))

In [ ]:
vidxtestdf.show(5)

+---+---------+------+------------+-------+--------------+----------+------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+
|age|workclass|weight|   education|edu num|marital status|occupation|relationship| race|   sex|captial-gain|capital-loss|hours-per-week|native country|label|            features|label_idx|
+---+---------+------+------------+-------+--------------+----------+------------+-----+------+------------+------------+--------------+--------------+-----+--------------------+---------+
| 17|        ?| 34505|        11th|      7| Never-married|         ?|   Own-child|White|  Male|           0|           0|            50| United-States|   No|[17.0,7.0,0.0,0.0...|      0.0|
| 17|        ?| 44789|Some-college|     10| Never-married|         ?|   Own-child|White|  Male|           0|           0|            15| United-States|   No|[17.0,10.0,0.0,0....|      0.0|
| 17|        ?| 80077|        11th|      7| Never-marri

In [ ]:
lrtestresult=lrmodel.evaluate(vidxtestdf)

In [ ]:
lrtestresult.accuracyk

0.8136160714285714

In [ ]:
lrtestresult.recallByLabel

[0.9490734568724171, 0.3821656050955414]

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
stdscl = StandardScaler(inputCol='features',outputCol='sclfeatures')

In [ ]:
stdsclmodel =stdscl.fit(vidxtraindf)

In [ ]:
stdsclmodel.mean

DenseVector([38.6334, 10.0662, 1090.7763, 86.6891, 40.4766])

In [ ]:
stdsclmodel.std

DenseVector([13.7056, 2.578, 7476.798, 401.0768, 12.3845])

In [ ]:
scltraindf=stdsclmodel.transform(vidxtraindf)

In [ ]:
scltraindf.select('features','sclfeatures').show()

+--------------------+--------------------+
|            features|         sclfeatures|
+--------------------+--------------------+
|[17.0,7.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,8.0,0.0,0.0...|[1.24037097913212...|
|[17.0,7.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,7.0,0.0,0.0...|[1.24037097913212...|
|[17.0,7.0,0.0,0.0...|[1.24037097913212...|
|[17.0,7.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,8.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.24037097913212...|
|[17.0,7.0,0.0,0.0...|[1.24037097913212...|
|[17.0,5.0,0.0,0.0...|[1.24037097913212...|
|[17.0,6.0,0.0,0.0...|[1.2403709

In [ ]:
newlr=LogisticRegression(featuresCol='features1',labelCol='labelidx1', maxIter=1000, tol=.0001,regParam=0)

In [ ]:
cstrain.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- edu num: integer (nullable = true)
 |-- marital status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- captial-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native country: string (nullable = true)
 |-- label: string (nullable = true)



In [ ]:
catcol =['marital statu','occupation','sex']

In [ ]:
catindexcol=[c+'_index' for c in catcol]

In [ ]:
catindexcol

['marital statu_index', 'occupation_index', 'sex_index']

In [ ]:
sicat = StringIndexer( inputCols = catcol, outputCols = catindexcol)

In [ ]:
sicat

StringIndexer_f5cf2f41ae3d

In [ ]:
cstrain.count()

39096

In [163]:
from pyspark.ml.pipeline import Pipeline

decision tree

In [149]:
from pyspark.ml.classification import DecisionTreeClassifier

In [161]:
from pyspark.ml.pipeline import Pipeline


In [162]:
dt=DecisionTreeClassifier(featuresCol='features', labelCol='label_index',maxDepth=30)

In [ ]:
dtp =Pipeline(stages=[va_scale_p, ss_p])